In [74]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio import features
from rasterio.enums import MergeAlg
from rasterio.plot import show
from numpy import int16
from PIL import Image
from rasterio.windows import Window
from rasterio import CRS
import numpy

In [2]:
# Open the GeoTIFF image
raster = rasterio.open("cog.tif")

In [3]:
labels_gdf = gpd.read_file("labels.geojson")

In [4]:
labels_gdf = labels_gdf.to_crs(raster.crs)

In [5]:
geom = [shapes for shapes in labels_gdf.geometry]

In [91]:
rasterized = features.rasterize(geom,
                                out_shape = raster.shape,
                                fill = 0,
                                out = None,
                                transform = raster.transform,
                                all_touched = True,
                                default_value = 1,
                                dtype = None)
label = numpy.expand_dims(rasterized, 0)

In [94]:

with rasterio.open('cog.tif') as src:
        
    kwargs = src.meta.copy()
    kwargs.update({'count':1})
    with rasterio.open(f'labels.tif','w', **kwargs) as dst:
        dst.write(label)

In [11]:
chipsize = 512
nrows = int(image.shape[1] / chipsize)
ncolumns = int(image.shape[2] / chipsize)
for row in range (nrows):
    for col in range (ncolumns):
        label_chip = rasterized[col*512:(col+1)*512,row*512:(row+1)*512]
        if label_chip.any():
            img_chip = image[:,col*512:(col+1)*512,row*512:(row+1)*512].transpose(1,2,0)
            label = Image.fromarray(label_chip)
            label.save(f"/home/data/solar/labels/solar_label_{col}_{row}.tif")
            img = Image.fromarray(img_chip)
            img.save(f"/home/data/solar/images/solar_image_{col}_{row}.tif")

In [102]:
chipsize = 512
nrows = int(image.shape[1] / chipsize)
ncolumns = int(image.shape[2] / chipsize)
for row in range (nrows):
    for col in range (ncolumns):
        win = Window(col*chipsize, row*chipsize, chipsize, chipsize)
        with rasterio.open('labels.tif') as lbl:
            label_chip = lbl.read(window=win)
            kwargs = lbl.meta.copy()
            kwargs.update({
                'height':chipsize,
                'width':chipsize,
                'transform':rasterio.windows.transform(win, lbl.transform),
                'crs':lbl.crs})
            if label_chip.any():
                with rasterio.open(f'/home/data/solar/labels/solar_{col}_{row}.tif','w', **kwargs) as dst:
                    dst.write(label_chip)
                kwargs.update({'count':3})
                with rasterio.open('cog.tif') as src:
                    image_chip = src.read(window=win)[:3]
                    with rasterio.open(f'/home/data/solar/images/solar_{col}_{row}.tif','w', **kwargs) as dst:
                        dst.write(image_chip)

In [100]:
with rasterio.open('/home/data/solar/images/solar_12_12.tif') as src:
    print (src.meta)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 512, 'height': 512, 'count': 4, 'crs': CRS.from_epsg(3857), 'transform': Affine(0.14929107086948487, 0.0, -7989083.540824194,
       0.0, -0.14929107086948487, 5205973.122446784)}


In [85]:
with rasterio.open('/home/data/kenya/images/kenol2_1710.tif') as src:
    print (src.read().shape)

(4, 512, 512)
